In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, RandomizedSearchCV
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score, mean_absolute_error

#Libraries for visualizing trees
from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image
import time as tm

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
binned_jobs = pd.read_csv("binned_jobs.csv")
binned_jobs.drop(['job_title', 'salary'], axis = 1)

# Convert categorical variables to dummy variables
binned_jobs_encoded = pd.get_dummies(binned_jobs, drop_first=True)

# Separate the features (X) and the target variable (y)
X = binned_jobs_encoded.drop(['salary_in_usd', 'salary'],axis=1)
y = binned_jobs_encoded['salary_in_usd']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [3]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from pyearth import Earth

# Define the parameter grid
param_grid = {
    'max_terms': [50,100, 200],  # Vary the max_terms values as needed
    'max_degree': [1, 2, 3]  # Vary the degree values as needed
}

# Initialize the MARS model
mars = Earth()

# Create the GridSearchCV object with MAE as the scoring metric
grid_search = GridSearchCV(mars, param_grid, scoring='neg_mean_absolute_error', cv=3)

# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

# Print the best model parameters and MAE
print("Best Model Parameters:")
print("max_terms:", grid_search.best_params_['max_terms'])
print("degree:", grid_search.best_params_['max_degree'])
print("MAE:", -grid_search.best_score_)  # Convert negative score back to positive


KeyboardInterrupt



In [ ]:
model3 = Earth(max_terms = grid_search.best_params_['max_terms'], max_degree=grid_search.best_params_['max_degree']) # note, terms in brackets are the hyperparameters
model3.fit(X_train,y_train)

# Predict on the test data using the best model
y_pred = model3.predict(X_test)

# Calculate the mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the mean absolute error
print("MAE: $", mae)

In [ ]:
# Convert y_pred array to a DataFrame
df_pred = pd.DataFrame({'y_pred': y_pred})

# Save the DataFrame to a CSV file
df_pred.to_csv('predictions.csv', index=False)

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor
from pyearth import Earth

# Initialize the Earth model
earth = Earth(max_terms = 50, max_degree=1)

# Initialize the BaggingRegressor with Earth as the base estimator
bagging_earth = BaggingRegressor(base_estimator=earth)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 250]  # Vary the number of estimators as needed
}

# Create the GridSearchCV object with MAE as the scoring metric
grid_search = GridSearchCV(bagging_earth, param_grid, scoring='neg_mean_absolute_error', cv=3)

# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

# Print the best model parameters and MAE
print("Best Model Parameters:")
print("n_estimators:", grid_search.best_params_['n_estimators'])
print("MAE:", -grid_search.best_score_)  # Convert negative score back to positive

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_absolute_error
from pyearth import Earth

# Create the MARS model
mars_model = Earth(max_terms = 50, max_degree=1)

# Create the bagging regressor with the MARS model
bagged_model = BaggingRegressor(base_estimator=mars_model, n_estimators=grid_search.best_params_['n_estimators'], random_state=42)

# Fit the bagged model to the training data
bagged_model.fit(X_train, y_train)

# Predict on the test data
y_pred = bagged_model.predict(X_test)

# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)

# Print the MAE
print("MAE: ", mae)

In [ ]:
# Convert y_pred array to a DataFrame
df_pred = pd.DataFrame({'y_pred': y_pred})

# Save the DataFrame to a CSV file
df_pred.to_csv('bagged_mars_predictions.csv', index=False)

In [ ]:
resids = y_test - preds
sns.scatterplot(x=X_test, y=preds)